In [1]:
!pip install "chronos-forecasting[training] @ git+https://github.com/amazon-science/chronos-forecasting.git"
!pip install peft transformers accelerate

  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-install-izn0thej/chronos-forecasting_55b3a4e4726e43828a94c175d2fd03c0
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-install-izn0thej/chronos-forecasting_55b3a4e4726e43828a94c175d2fd03c0
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit 94e20ea7e510ac4d665492b8bed8836a5143f16e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
from pathlib import Path
from gluonts.dataset.arrow import ArrowWriter
from chronos import ChronosPipeline
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import torch
from tqdm.auto import tqdm
import time
import yaml
from datetime import datetime

In [31]:
import yfinance as yf
import pandas as pd

def fetch_financial_data():
    data_dict = {}

    def download_close(symbol, start_date='1928-01-01', end_date='2025-01-02'):
        # Download the data
        df = yf.download(symbol, start=start_date, end=end_date, interval="1d", progress=False)

        if df.empty:
            return None

        # -- 1. Flatten multi-level columns if necessary --
        # Sometimes yfinance returns multi-level columns, especially if there's a grouping.
        if isinstance(df.columns, pd.MultiIndex):
            # Flatten the columns into strings like 'Close' or 'Close_Symbol'
            df.columns = ['_'.join(tuple(str(level) for level in col if level))
                          for col in df.columns]
            # After flattening, the 'Close' column might appear as 'Close' or 'Close_^GSPC', etc.
            # We'll look for any flattened column name starting with 'Close'.
            potential_close_cols = [c for c in df.columns if c.lower().startswith('close')]
            if len(potential_close_cols) == 1:
                # Found a unique column for 'Close'
                close_col = potential_close_cols[0]
            else:
                print(f"WARNING: Multiple or zero 'Close' columns found for {symbol}: {potential_close_cols}")
                return None
        else:
            # Single-level columns, just look for 'Close'
            if 'Close' not in df.columns:
                print(f"WARNING: No 'Close' column found for {symbol}")
                return None
            close_col = 'Close'

        # -- 2. Reset index to be sure 'Date' is a column, then re-set it as a DatetimeIndex --
        df = df.reset_index(drop=False)
        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
            df = df.set_index('Date')
        else:
            # If there's no 'Date' column, we can't proceed
            print(f"WARNING: No 'Date' column available for {symbol}.")
            return None

        # -- 3. Extract the 'Close' column as a real Series (1-D) --
        # Using `.squeeze()` if it's 1-D ensures it's truly a Series.
        close_series = df[[close_col]].squeeze(axis=1)
        # At this point, close_series should be a Pandas Series indexed by Date.

        # If for some reason it’s not a Series:
        if not isinstance(close_series, pd.Series):
            print(f"WARNING: {symbol} 'Close' extraction did not return a Series.")
            return None

        # Return the close prices
        return close_series

    # =====================
    # 1) S&P 500
    # =====================
    print("Fetching S&P 500 data...")
    try:
        sp500_close = download_close('^GSPC', start_date='1928-01-01', end_date='2025-01-02')
        if sp500_close is not None:
            data_dict['SP500'] = sp500_close
            print(f"Added SP500 with {len(sp500_close)} entries")
    except Exception as e:
        print(f"Error fetching S&P 500: {e}")

    # # =====================
    # # 2) Forex
    # # =====================
    # print("\nFetching Forex data...")
    # forex_pairs = {
    #     'EUR=X': 'EURUSD',
    #     'GBP=X': 'GBPUSD',
    #     'JPY=X': 'JPYUSD',
    #     'AUD=X': 'AUDUSD',
    #     'CAD=X': 'CADUSD'
    # }
    # for symbol, name in forex_pairs.items():
    #     try:
    #         fx_close = download_close(symbol)
    #         if fx_close is not None:
    #             data_dict[name] = fx_close
    #             print(f"Added {name} with {len(fx_close)} entries")
    #     except Exception as e:
    #         print(f"Failed to fetch {name}: {e}")

    # # =====================
    # # 3) Commodities
    # # =====================
    # print("\nFetching Commodities data...")
    # commodities = {
    #     'GC=F': 'GOLD',
    #     'SI=F': 'SILVER',
    #     'CL=F': 'CRUDE_OIL',
    #     'NG=F': 'NAT_GAS',
    #     'ZC=F': 'CORN',
    #     'ZW=F': 'WHEAT',
    #     'HG=F': 'COPPER'
    # }
    # for symbol, name in commodities.items():
    #     try:
    #         comm_close = download_close(symbol)
    #         if comm_close is not None:
    #             data_dict[name] = comm_close
    #             print(f"Added {name} with {len(comm_close)} entries")
    #     except Exception as e:
    #         print(f"Failed to fetch {name}: {e}")

    # # =====================
    # # 4) Crypto
    # # =====================
    # print("\nFetching Crypto data...")
    # cryptos = {
    #     'BTC-USD': ('BTC', '2015-01-01'),
    #     'ETH-USD': ('ETH', '2017-01-01'),
    #     'BNB-USD': ('BNB', '2017-01-01'),
    #     'XRP-USD': ('XRP', '2017-01-01'),
    #     'SOL-USD': ('SOL', '2020-01-01')
    # }
    # for symbol, (name, start) in cryptos.items():
    #     try:
    #         crypto_close = download_close(symbol, start_date=start)
    #         if crypto_close is not None:
    #             data_dict[name] = crypto_close
    #             print(f"Added {name} with {len(crypto_close)} entries")
    #     except Exception as e:
    #         print(f"Failed to fetch {name}: {e}")

    # =====================
    # Verify we got any data
    # =====================
    if not data_dict:
        raise ValueError("No data was successfully collected.")

    # =====================x
    # Process each series
    # =====================
    processed_series = []

    for name, series in data_dict.items():
        # Double-check it's really a Series
        if not isinstance(series, pd.Series):
            print(f"WARNING: '{name}' is not a Series, skipping.")
            continue

        # Convert to numeric (coerce any non-numeric to NaN)
        clean_series = pd.to_numeric(series, errors='coerce')

        # Forward-fill, then back-fill, then drop any leftover NaN
        clean_series = clean_series.ffill().bfill().dropna()

        if len(clean_series) > 0:
            clean_series.name = name
            processed_series.append(clean_series)
            print(f"Processed {name}: {len(clean_series)} entries "
                  f"from {clean_series.index[0].strftime('%Y-%m-%d')} "
                  f"to {clean_series.index[-1].strftime('%Y-%m-%d')}")

    return processed_series

series_list = fetch_financial_data()

Fetching S&P 500 data...
Added SP500 with 24366 entries
Processed SP500: 24366 entries from 1928-01-03 to 2024-12-31


In [32]:
def split_time_series(series, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    """
    Splits a time series into train, validation, and test sets while preserving time order.
    """
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1."

    n = len(series)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))

    train_data = series.iloc[:train_end]
    val_data = series.iloc[train_end:val_end]
    test_data = series.iloc[val_end:]

    return train_data, val_data, test_data

In [33]:
series_list

[Date
 1928-01-03      17.760000
 1928-01-04      17.719999
 1928-01-05      17.549999
 1928-01-06      17.660000
 1928-01-09      17.500000
                  ...     
 2024-12-24    6040.040039
 2024-12-26    6037.589844
 2024-12-27    5970.839844
 2024-12-30    5906.939941
 2024-12-31    5881.629883
 Name: SP500, Length: 24366, dtype: float64]

In [34]:
# Apply split for each series in series_list
train_series = []
val_series = []
test_series = []

for series in series_list:
    train_data, val_data, test_data = split_time_series(series)
    train_series.append(train_data)
    val_series.append(val_data)
    test_series.append(test_data)

    print(f"Splitted {series.name} -> Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")

Splitted SP500 -> Train: 19492, Val: 2437, Test: 2437


In [17]:
# Ensure the first element of series_list is used correctly
returns_clipped = series_list[0].clip(lower=-0.1, upper=0.1)

# Handle NaN values before min-max scaling
returns_clipped = returns_clipped.fillna(method="ffill")

# Compute min-max values with NaN handling
min_val = returns_clipped.min(skipna=True)
max_val = returns_clipped.max(skipna=True)


# Ensure normalization only occurs if min/max are valid
if pd.isna(min_val) or pd.isna(max_val):
    print("Warning: Min/Max values are NaN, check input data!")
else:
    normalized_series = (returns_clipped - min_val) / (max_val - min_val)

print(normalized_series.head())  # Verify results

Date
1928-01-03   NaN
1928-01-04   NaN
1928-01-05   NaN
1928-01-06   NaN
1928-01-09   NaN
Name: SP500, dtype: float64


<ipython-input-17-5f05f605d91c>:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  returns_clipped = returns_clipped.fillna(method="ffill")


In [18]:
print(returns_clipped.isna().sum())  # Count NaNs
print(f"Min Value: {min_val}, Max Value: {max_val}")  # Check min/max values
print(returns_clipped.head())  # Preview first few values

0
Min Value: 0.1, Max Value: 0.1
Date
1928-01-03    0.1
1928-01-04    0.1
1928-01-05    0.1
1928-01-06    0.1
1928-01-09    0.1
Name: SP500, dtype: float64


In [12]:
normalized_series

,SP500
Date,
1928-01-03,NaN
1928-01-04,NaN
1928-01-05,NaN
1928-01-06,NaN
1928-01-09,NaN
...,...
2022-12-23,NaN
2022-12-27,NaN
2022-12-28,NaN


In [7]:
def preprocess_data(series):
    """
    Apply simplified preprocessing steps:
    1. Convert to returns (percent changes)
    2. Clip extreme values
    3. Normalize data
    """
    # Ensure we're working with a Series
    if not isinstance(series, pd.Series):
        print("Warning: Input is not a Series. Converting...")
        series = pd.Series(series.values.flatten(), index=series.index, name='SP500')

    # Convert to returns (percent changes) as per paper recommendation
    returns = series

    # Clip extreme values (outliers)
    returns_clipped = returns.clip(lower=-0.1, upper=0.1)

    # Min-max scaling for normalization
    min_val = returns_clipped.min()
    max_val = returns_clipped.max()
    normalized_returns = (returns_clipped - min_val) / (max_val - min_val)

    # Preserve the name of the original series
    try:
        normalized_returns.name = series.name
    except:
        normalized_returns.name = 'SP500'

    return normalized_returns

In [8]:
processed_series = [preprocess_data(series) for series in series_list]

In [9]:
processed_series

[Date
 1928-01-03   NaN
 1928-01-04   NaN
 1928-01-05   NaN
 1928-01-06   NaN
 1928-01-09   NaN
               ..
 2022-12-23   NaN
 2022-12-27   NaN
 2022-12-28   NaN
 2022-12-29   NaN
 2022-12-30   NaN
 Name: SP500, Length: 23864, dtype: float64]

In [19]:
series_list

[Date
 1928-01-03      17.760000
 1928-01-04      17.719999
 1928-01-05      17.549999
 1928-01-06      17.660000
 1928-01-09      17.500000
                  ...     
 2022-12-23    3844.820068
 2022-12-27    3829.250000
 2022-12-28    3783.219971
 2022-12-29    3849.280029
 2022-12-30    3839.500000
 Name: SP500, Length: 23864, dtype: float64]

In [20]:
# Extract raw price series
raw_prices = series_list[0]  # Since it's a Series

# Compute min and max values
min_val = raw_prices.min()
max_val = raw_prices.max()

# Ensure min and max are not equal to avoid division by zero
if min_val == max_val:
    print("Warning: All values are the same. Normalization is not possible.")
    normalized_series = raw_prices  # Keep as is
else:
    normalized_series = (raw_prices - min_val) / (max_val - min_val)

# Check results
print(normalized_series.head())

Date
1928-01-03    0.002788
1928-01-04    0.002780
1928-01-05    0.002744
1928-01-06    0.002767
1928-01-09    0.002734
Name: SP500, dtype: float64


In [37]:
train_series

[Date
 1928-01-03      17.760000
 1928-01-04      17.719999
 1928-01-05      17.549999
 1928-01-06      17.660000
 1928-01-09      17.500000
                  ...     
 2005-08-12    1230.390015
 2005-08-15    1233.869995
 2005-08-16    1219.339966
 2005-08-17    1220.239990
 2005-08-18    1219.020020
 Name: SP500, Length: 19492, dtype: float64]

In [38]:
# Extract raw price series
raw_prices_train = train_series[0]  # Since it's a Series

# Compute min and max values
min_val_train = raw_prices_train.min()
max_val_train = raw_prices_train.max()

train_normalized_series = (raw_prices_train - min_val_train) / (max_val_train - min_val_train)

# Check results
print(train_normalized_series.head())

Date
1928-01-03    0.008772
1928-01-04    0.008746
1928-01-05    0.008634
1928-01-06    0.008706
1928-01-09    0.008601
Name: SP500, dtype: float64


In [39]:
# Extract raw price series
raw_prices_val = val_series[0]  # Since it's a Series

# Compute min and max values
min_val_val = raw_prices_val.min()
max_val_val = raw_prices_val.max()

val_normalized_series = (raw_prices_val - min_val_val) / (max_val_val - min_val_val)

# Check results
print(val_normalized_series.head())

Date
2005-08-19    0.376905
2005-08-22    0.378306
2005-08-23    0.375434
2005-08-24    0.369883
2005-08-25    0.371812
Name: SP500, dtype: float64


In [40]:
from chronos import ChronosPipeline

pipeline = ChronosPipeline.from_pretrained("amazon/chronos-t5-tiny")
model = pipeline.model.model  # Extract base model
tokenizer = pipeline.tokenizer  # Get tokenizer

In [22]:
### ---- first run --- ###

from peft import LoraConfig, get_peft_model

# Define LoRA configuration
lora_config = LoraConfig(
    r=2,  # LoRA rank
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,
    target_modules=["SelfAttention.q", "SelfAttention.k", "SelfAttention.v", "SelfAttention.o",
                    "EncDecAttention.q", "EncDecAttention.k", "EncDecAttention.v", "EncDecAttention.o"],
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

# Freeze all model parameters
for param in model.parameters():
    param.requires_grad = False

# Apply LoRA
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 49,152 || all params: 8,443,648 || trainable%: 0.5821


In [41]:
# ### --- second --- ###

# from peft import LoraConfig, get_peft_model

# # Define LoRA configuration
# lora_config = LoraConfig(
#     r=4,  # LoRA rank
#     lora_alpha=16,  # Scaling factor
#     lora_dropout=0.3,
#     bias="none",
#     task_type="SEQ_2_SEQ_LM"
# )

# # Apply LoRA
# lora_model = get_peft_model(model, lora_config)
# lora_model.print_trainable_parameters()

In [ ]:
# ### --- third --- ###

# from peft import LoraConfig, get_peft_model

# # Define LoRA configuration
# lora_config = LoraConfig(
#     r=6,  # LoRA rank
#     lora_alpha=32,  # Scaling factor
#     lora_dropout=0.1,
#     bias="none",
#     task_type="SEQ_2_SEQ_LM"
# )

# # Apply LoRA
# lora_model = get_peft_model(model, lora_config)
# lora_model.print_trainable_parameters()

trainable params: 663,552 || all params: 202,038,528 || trainable%: 0.3284


In [42]:
pipeline = ChronosPipeline.from_pretrained("amazon/chronos-t5-tiny")

# Load dataset into Chronos
training_data = "./financial_data.arrow"
val_data = "./financial_val_data.arrow"

In [43]:
!git clone https://github.com/amazon-science/chronos-forecasting.git

Cloning into 'chronos-forecasting'...
remote: Enumerating objects: 466, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 466 (delta 2), reused 0 (delta 0), pack-reused 462 (from 2)
Receiving objects: 100% (466/466), 1.01 MiB | 29.51 MiB/s, done.
Resolving deltas: 100% (198/198), done.


In [44]:
cd chronos-forecasting

/content/chronos-forecasting/chronos-forecasting


In [45]:
from gluonts.dataset.arrow import ArrowWriter
import pandas as pd
import numpy as np

def convert_financial_series_to_arrow(series, output_path="./financial_data.arrow"):
    dataset = []
    for series in series_list:
        if not isinstance(series, pd.Series):
            print(f"Skipping non-series data: {series}")
            continue

        series = pd.to_numeric(series, errors='coerce').dropna()
        if not np.issubdtype(series.index.dtype, np.datetime64):
            series.index = pd.to_datetime(series.index)

        dataset.append({"start": str(series.index[0]), "target": series.values.tolist()})

    ArrowWriter(compression="lz4").write_to_file(dataset, path=output_path)
    print(f"✅ Financial data saved to {output_path}")

In [47]:
convert_financial_series_to_arrow(train_normalized_series, output_path="./financial_data.arrow")

✅ Financial data saved to ./financial_data.arrow


In [48]:
convert_financial_series_to_arrow(val_normalized_series, output_path="./financial_val_data.arrow")

✅ Financial data saved to ./financial_val_data.arrow


In [ ]:
# %%writefile ./config.yaml
# training_data_paths:
#   - "./financial_data.arrow"
# probability:
#   - 1.0
# model:
#   model_id: "amazon/chronos-t5-tiny"
#   random_init: false
# training:
#   max_steps: 10000
#   learning_rate: 5e-5
#   lr_scheduler_type: "cosine"
#   batch_size: 16
#   gradient_accumulation_steps: 4
#   warmup_ratio: 0.05
#   weight_decay: 0.01
#   fp16: true
#   gradient_checkpointing: true
#   save_steps: 5000
#   logging_steps: 100
#   eval_steps: 5000
#   early_stopping_patience: 3
# lora:
#   r: 2
#   alpha: 16
#   dropout: 0.1
#   bias: "none"
#   target_modules: ["SelfAttention.q", "SelfAttention.k", "SelfAttention.v", "SelfAttention.o",
#                   "EncDecAttention.q", "EncDecAttention.k", "EncDecAttention.v", "EncDecAttention.o"]

Overwriting ./config.yaml


In [53]:
%%writefile ./config.yaml
training_data_paths:
  - "./financial_data.arrow"
validation_data_paths:  # ✅ Added validation dataset
  - "./financial_val_data.arrow"
probability:
  - 1.0
model:
  model_id: "amazon/chronos-t5-tiny"
  random_init: false
training:
  max_steps: 3000  # ✅ Reduced steps to prevent overfitting
  learning_rate: 5e-5
  lr_scheduler_type: "cosine"
  batch_size: 16
  gradient_accumulation_steps: 4
  warmup_ratio: 0.05
  weight_decay: 0.01
  fp16: true
  gradient_checkpointing: true
  save_steps: 500  # ✅ Save checkpoints more frequently (500 instead of 5000)
  logging_steps: 100
  eval_steps: 500  # ✅ More frequent validation evaluation
  evaluation_strategy: "steps"  # ✅ Ensure validation runs every eval_steps
  save_strategy: "epoch"  # ✅ Save best checkpoint at the end of each epoch
  load_best_model_at_end: true  # ✅ Automatically use best-performing model
  metric_for_best_model: "eval_loss"  # ✅ Track validation loss
  greater_is_better: false  # ✅ Lower loss is better
  early_stopping_patience: 3  # ✅ Stop training if no improvement for 3 eval cycles
lora:
  r: 2
  alpha: 16
  dropout: 0.1
  bias: "none"
  target_modules: ["SelfAttention.q", "SelfAttention.k", "SelfAttention.v", "SelfAttention.o",
                  "EncDecAttention.q", "EncDecAttention.k", "EncDecAttention.v", "EncDecAttention.o"]


Overwriting ./config.yaml


In [ ]:
# # --- 2nd --- #

# %%writefile ./config.yaml
# training_data_paths:
#   - "./financial_data.arrow"
# probability:
#   - 1.0

# model:
#   model_id: "amazon/chronos-t5-base"
#   random_init: false

# training:
#   max_steps: 20000  # Reduced steps to avoid overfitting; can increase to 35000 if needed
#   learning_rate: 5e-5  # Lower LR to prevent catastrophic forgetting
#   lr_scheduler_type: "cosine"  # Stabilizes learning
#   batch_size: 8  # Smaller batch size for more frequent updates
#   gradient_accumulation_steps: 4  # Effective batch size = 32
#   warmup_ratio: 0.1
#   weight_decay: 0.05  # Increased to prevent overfitting
#   fp16: true
#   gradient_checkpointing: true  # Stabilizes training
#   save_steps: 5000  # Save every ~2.5 epochs
#   logging_steps: 500  # Log progress every 500 steps
#   eval_steps: 1333  # Evaluate every ~1 epoch
#   early_stopping_patience: 3  # Stop if no improvement for 3 evaluations

# lora:
#   rank: 4  # Reduced to prevent over-adaptation
#   alpha: 16  # Keeps scaling factor same
#   dropout: 0.3  # Increased to avoid overfitting
#   bias: "none"
#   task_type: "SEQ_2_SEQ_LM"

Writing ./config.yaml


In [ ]:
# %%writefile ./config.yaml
# training_data_paths:
#   - "./financial_data.arrow"
# probability:
#   - 1.0

# model:
#   model_id: "amazon/chronos-t5-base"
#   random_init: false

# training:
#   max_steps: 31127  # Adjusted for 20 epochs
#   learning_rate: 3e-5  # Kept LR conservative to avoid overfitting
#   lr_scheduler_type: "cosine"
#   batch_size: 8
#   gradient_accumulation_steps: 4
#   warmup_ratio: 0.1
#   weight_decay: 0.005
#   fp16: true
#   gradient_checkpointing: true
#   save_steps: 5000
#   logging_steps: 500
#   eval_steps: 1500
#   early_stopping_patience: 3

# lora:
#   rank: 6
#   alpha: 32  # Increased from 16 → 24 for stronger adaptation
#   dropout: 0.1
#   bias: "none"
#   task_type: "SEQ_2_SEQ_LM"

Writing ./config.yaml


In [ ]:
# %%writefile ./config.yaml
# training_data_paths:
#   - "./financial_data.arrow"
# probability:
#   - 1.0

# model:
#   model_id: "amazon/chronos-t5-base"
#   random_init: false

# training:
#   max_steps: 31127  # Keep around 30k steps
#   learning_rate: 2e-5  # Slightly reduced learning rate for stability
#   lr_scheduler_type: "cosine"
#   batch_size: 8  # Keep batch size the same
#   gradient_accumulation_steps: 4  # Effective batch size remains 32
#   warmup_ratio: 0.1
#   weight_decay: 0.01  # Increased to regularize training
#   fp16: true
#   gradient_checkpointing: true  # Helps with memory efficiency
#   save_steps: 5000
#   logging_steps: 500
#   eval_steps: 1500
#   early_stopping_patience: 3  # Keeps early stopping in place

In [ ]:
pwd

'/content/chronos-forecasting/chronos-forecasting/chronos-forecasting'

In [ ]:
ls

ci/                 config.yaml      financial_data.arrow  NOTICE          scripts/
CITATION.cff        CONTRIBUTING.md  LICENSE               pyproject.toml  src/
CODE_OF_CONDUCT.md  figures/         notebooks/            README.md       test/


In [ ]:
import torch
print(torch.cuda.is_available())

True


In [58]:
# Train the model - explicitly specifying the same parameters as in the config file
!CUDA_VISIBLE_DEVICES=0 python scripts/training/train.py \
    --config ./config.yaml \
    --model-id amazon/chronos-t5-tiny \
    --no-random-init \
    --max-steps 3000 \
    --learning-rate 5e-5 \
    --lr-scheduler-type cosine \
    --gradient-accumulation-steps 4 \
    --warmup-ratio 0.05 \
    --save-steps 500 \

# Echo training completion
!echo "Training finished. Checking for checkpoints..."

# List the output directory to confirm checkpoints were created
!ls -la output/run-0/

# Zip the fine-tuned model (run-0) and rename it
!zip -r fine_tuned_chronos_sp500_rk2_3000.zip output/run-0/

# Save the zip file to Google Drive
!cp fine_tuned_chronos_sp500_rk2_3000.zip /content/drive/MyDrive/

2025-03-04 04:43:33.857362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741063413.879172   23620 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741063413.885877   23620 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-04 04:43:37,268 - /content/chronos-forecasting/chronos-forecasting/scripts/training/train.py - INFO - Using SEED: 727628000
2025-03-04 04:43:37,274 - /content/chronos-forecasting/chronos-forecasting/scripts/training/train.py - INFO - Logging dir: output/run-0
2025-03-04 04:43:37,274 - /content/chronos-forecasting/chronos-forecasting/scripts/training/train.py - INFO - Loading and filtering 1 datasets for training: ['./financia

In [ ]:
ls output/run-0

checkpoint-27330/  checkpoint-final/  logs/


In [ ]:
!mkdir -p fine_tuned_chronos
!cp -r output/run-0/* fine_tuned_chronos/

In [ ]:
ls

ci/                 config.yaml      financial_data.arrow  NOTICE          README.md  test/
CITATION.cff        CONTRIBUTING.md  fine_tuned_chronos/   output/         scripts/
CODE_OF_CONDUCT.md  figures/         LICENSE               pyproject.toml  src/
